In [ ]:
from pandas import Series,DataFrame
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.font_manager as fm
from matplotlib import pylab
import datetime as dt
import pandas as pd
import calendar
from multiprocessing import Pool
import multiprocessing
import time
from glob import glob
import numpy as np

import seaborn as sns
#데이터시각화 도구(설치필요)
sns.set_style("whitegrid")
sns.set_context("paper")
#설정


%matplotlib inline
count=0


########################csv파일 읽어오고 dataframe생성#####################

start_time = time.time()  # 시간측정시작
df = pd.DataFrame()       # 비어있는 dataframe생성
folder = "./dataset/"     # dataset 폴더의 경로
file_list = glob(folder + '*.csv')  #dataset 폴더에 들어있는 csv 확장자 리스트를 전부 가져온다.
for file in file_list:             #csv파일들을 하나의 dataframe으로 연결
    count+=1
    print("{0}번째 파일 읽는중..".format(count))
    f = pd.read_csv(file, engine='python',error_bad_lines=False, warn_bad_lines=False)
    df = pd.concat([df,f]) 
    print("경과 시간 : %s 초 " %((time.time() - start_time)))
print("데이터프레임 연결까지 경과 시간 : %s 분 " %((time.time() - start_time)/60))

########어종 = '가자미류'에 해당하는 row를 가져와 kg당가격을 구한다########

#어종상태명이 '(활)' 인 row의 해당 속성 열을 가져온다.
df = df.loc[df.어종상태명=="(활)",["위판일자","수산물표준코드명","산지조합명","위판중량", "위판금액"]]
#코드명이 '가자미류'인 row만 선택한다.
df = df.loc[df.수산물표준코드명=="가자미류",["위판일자","산지조합명","위판중량", "위판금액"]]



df = df[~(df == 0).any(axis=1)]
#열에 있는 데이터중에 0인 데이터를 지운다 . axis = 1 은 그 해당 행을 지우고
#                                           axis = 0 은 그 해당 열을 지운다.
#0 value를 포함한 행을 삭제한다
#데이터의 갯수가 충분히 많으므로 0인 데이터를 뺴줘도 무관하다.
#데이터의 갯수가 충분하지 않다면 행삭제를 고려할 것


def sublst(row):
    kg = int(round((row['위판금액']/row['위판중량'])))

    return kg
#"kg당가격"이라는 비어있는 column을 추가하고 위판금액을 위판중량으로 나눈 값을 저장

df['kg당가격'] = df.apply(sublst,axis=1)

#일자별 위판별 KG당 가격으로 데이터 전처리
#'kg당가격' calumun
#해양수산데이터에서 KG당 가격을 제공하지만 해당 데이터가 확인중 불확실하다. 
#따라서 전체 위판금액에서 위판중량을 나눠서 KG당 가격을 다시 구한다.

#'kg당가격으로 이상치 탐지를 작업합니다.'



#####################이상치제거 및 데이터 가공 부분#######################
#한글 폰트 지정
plt.rcParams["font.family"] = 'NanumBarunGothic'
plt.rcParams["font.size"] = 10


kp = 'kg당가격'

'''
plt.figure(figsize=(10,8))
plt.subplot(211)
plt.xlim(df[kp].min(), df[kp].max()*1.1)
ax = df[kp].plot(kind='kde')

plt.subplot(212)
plt.xlim(df[kp].min(), df[kp].max()*1.1)
sns.boxplot(x=df[kp])
'''
############################################################################

print(df)


df.loc[df.kg당가격 == 0, 'kg당가격'] = np.nan
#0의 값이 들어있는 데이터를 Nan으로 치환(0값은 로그변환시 에러 발생한다.)

df.dropna(inplace=True)
#dropna함수를 이용해 결측값이 포함된 행을 전체제거해준다.

df['Log_'+ kp] = np.log(df[kp])
#'Log_kg당가격'행'에 로그변환

q75, q25 = np.percentile(df.Log_kg당가격.dropna(), [75 ,25])
iqr = q75 - q25

min = q25 - (iqr*1.5)
max = q75 + (iqr*1.5)
#1.5를 곱해줘서 그 밖의 값은 이상치로 판단한다.


print(iqr)
print(q75)
print(q25)
#percentile함수로 백분위수 25~75에 해당하는 값을 구해준다.




kp = 'Log_kg당가격'

'''
plt.figure(figsize=(10,8))
plt.subplot(211)
plt.xlim(df[kp].min(), df[kp].max()*1.1)
ax = df[kp].plot(kind='kde')

plt.subplot(212)
plt.xlim(df[kp].min(), df[kp].max()*1.1)
sns.boxplot(x=df[kp])
'''


df['Outlier'] = 1

df.loc[df[kp] < min, 'Outlier'] = 0
df.loc[df[kp] > max, 'Outlier'] = 0

df.loc[df.Outlier == 0, 'Outlier'] = np.nan
#0의 값이 들어있는 데이터를 Nan으로 치환(0값은 로그변환시 에러 발생한다.)

df.dropna(inplace=True)
#dropna함수를 이용해 결측값이 포함된 행을 전체제거해준다.

del df["Outlier"]
#이상치판별 열 제거

###################위판일자별 kg당가격의 평균을 구한다######################

#그룹별 평균을 구한다. 여기서는 위판일자별 kg당가격 평균
df_mean = df.groupby(['위판일자'], as_index=False).mean() 
print(df_mean.to_string())

###############################그래프 생성##################################
#그래프 월별 색깔 지정 (10가지 색상)
prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']

#x,y좌표 라벨, 표 크기 지정
plt.figure(figsize=(24,10))
plt.xlabel('날짜')
plt.ylabel('가격(원)')

#[plot data] x좌표 : 날짜, y좌표 : 일 평균 가격
x,y = [],[]
for line in df_mean.위판일자:
    times = dt.datetime.strptime(str(line),'%Y%m%d')
    x.append(times)
for line in df_mean.kg당가격:
    y.append(line)
'''
#강조 구간
for i in range(1,13):
    span_start = dt.datetime(2013, i, 1)
    end_day = calendar.monthrange(2013, i)[1] # i월달의 마지막 일을 가져온다.
    span_end = dt.datetime(2013, i, end_day)
    plt.axvspan(span_start, span_end, facecolor=colors[i%10], alpha=0.5)

'''    
    
plt.title("가자미류 13년도 일 평균 가격(kg당)")
plt.plot(x,y,linestyle='-',c="m" ,label="value")
plt.show()
print("총 경과 시간 : %s 분 " %((time.time() - start_time)/60))


1번째 파일 읽는중..
경과 시간 : 12.4818115234375 초 
2번째 파일 읽는중..
경과 시간 : 24.49539828300476 초 
3번째 파일 읽는중..
경과 시간 : 38.137845516204834 초 
4번째 파일 읽는중..
경과 시간 : 49.86920976638794 초 
5번째 파일 읽는중..
경과 시간 : 58.703941106796265 초 
6번째 파일 읽는중..
경과 시간 : 70.06776976585388 초 
7번째 파일 읽는중..
경과 시간 : 84.09709644317627 초 
8번째 파일 읽는중..
경과 시간 : 101.88411068916321 초 
9번째 파일 읽는중..
경과 시간 : 118.00308012962341 초 
10번째 파일 읽는중..
경과 시간 : 131.49667358398438 초 
11번째 파일 읽는중..
경과 시간 : 149.6725504398346 초 
12번째 파일 읽는중..
